In [22]:
# Download GPT-4All model file
!wget https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin

--2023-08-25 08:07:40--  https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin
Resolving gpt4all.io (gpt4all.io)... 172.67.71.169, 104.26.1.159, 104.26.0.159, ...
Connecting to gpt4all.io (gpt4all.io)|172.67.71.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3785248281 (3.5G)
Saving to: ‘ggml-gpt4all-j-v1.3-groovy.bin.2’

ggml-gpt4all-j-v1.3 100%[===================>]   3.52G  57.9MB/s    in 82s     

2023-08-25 08:09:03 (44.2 MB/s) - ‘ggml-gpt4all-j-v1.3-groovy.bin.2’ saved [3785248281/3785248281]



In [23]:
pip install langchain

In [24]:
pip install gpt4all

In [25]:
pip install pypdf

In [26]:
pip install chromadb

In [27]:
pip install pdf2image

In [28]:
pip install sentence_transformers

In [29]:
# Import necessary packages
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from google.colab import drive
import time


In [30]:
# Mounts your Google Drive to the '/content/drive' directory in Colab
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
file_path = "/content/drive/MyDrive/GPT4All/capacity-building-framework-on-data-analytics_en.pdf"

In [32]:
# Load the Capacity building pdf file and print out numbers of pages and split the pdf into single pagesprint(documents[0].page_content)
loader = PyPDFLoader(file_path)
documents = loader.load_and_split()
len(documents)

69

In [33]:
print(documents[1].page_content)

1 Executive summary  
 
 
The amount of data being captured and stored by Customs today is overwhelming, especially with the 
influx of new technologies and the introduction of new Customs clearance systems which facilitate 
the recording of import, export  and transit transactions between countries . By discovering 
associations and understanding patterns and trends within this data, data analytics has the potential 
to improve the way Customs  deal with evolving threats, predict and manage non -compliance , and 
increase productivity and performance . 
 
However, implementing data analytics in Customs is not an easy task ; it requires considerable efforts 
to develop the appropriate organizational  and technical capabilities needed  to take advantage of the 
explosion in data , and to gain the insights that will  help make  better informed decisions.  
 
This framework uses the results of a recent WCO survey on data analytics in Customs  to provide 
information on how to effectively 

In [36]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=64
)
texts = text_splitter.split_documents(documents)
len(texts)

201

In [37]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [38]:
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

In [39]:
llm = GPT4All(
    model="./ggml-gpt4all-j-v1.3-groovy.bin",
    backend="gptj",
    verbose=False
)

Found model file at  ./ggml-gpt4all-j-v1.3-groovy.bin


In [40]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    verbose=False,
)

In [41]:
start_time = time.time()

res = qa(f"""
    What is the capacity building framework from the WCO for Data analytics about?
""")
print(res["result"])

end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

 The Capacity Building Framework (CBF) approved by the 11th Session of the Policy Commission in WCO CAPABILITY BUILDING FRAME WORK ON DATA ANALYTIC S, aims to provide Members with a comprehensive understanding and skillset on how to effectively implement data analytics initiatives within Customs. It is intended as an additional resource for addressing common obstacles related to implementing successful analytics projects while complementing the high-level overview of data analytics provided in the WCO Handbook on Data Analysis by supplying further details on technical, organizational, and strategic aspects necessary for achieving success with a data analytics strategy aligned with overall customs strategies.
Elapsed time: 4747.38 seconds
